# GREAT CARIA v3.0 - Relatividad General Financiera

## La Gran Teoría Unificada: Espacio, Tiempo e Incertidumbre

### Marco Teórico

**El sistema financiero es un espacio-tiempo curvo donde:**
- **Geografía** (G7 vs EM) = Espacio
- **Escalas temporales** (Fast/Medium/Slow) = Tiempo
- **Incertidumbre H** = Energía de desacople

### La Ecuación Maestra

$$\mu_{MSFI}(t) = \sum_{i} w_i \cdot s_i(t) \quad \text{(Media ponderada)}$$

$$H_{temp}(t) = \sqrt{\sum_{i} w_i (s_i - \mu)^2} \quad \text{(Dispersión temporal)}$$

$$\Delta_{geo}(t) = |\sigma^2_{G7} - \sigma^2_{EM}| \quad \text{(Desacople espacial)}$$

$$H_{total} = H_{temp} + \lambda \cdot \Delta_{geo} \quad \text{(Incertidumbre total)}$$

### La Predicción

- **H alto** → Sistema desacoplado → Robusto
- **H → 0** → Sincronización total → CRISIS

In [ ]:
# === SETUP ===
!pip install PyWavelets scikit-learn -q

import pandas as pd
import numpy as np
from scipy import stats, signal
import pywt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

print('='*60)
print('GREAT CARIA v3.0')
print('Relatividad General Financiera: Espacio × Tiempo × Incertidumbre')
print('='*60)

In [ ]:
# === LOAD DATA ===
DATA_PATH = '/content/drive/MyDrive/Caria/yahoo_market.parquet'
df = pd.read_parquet(DATA_PATH)
df.index = pd.to_datetime(df.index)

# Geographic blocks
G7 = ['USA', 'GBR', 'DEU', 'FRA', 'JPN', 'CAN']  # + ITA but may not be in data
EM = ['CHN', 'BRA', 'IND', 'MEX', 'KOR', 'ZAF', 'AUS']

# Get available countries
all_countries = [c.replace('_index', '') for c in df.columns if '_index' in c]
g7_avail = [c for c in G7 if c in all_countries]
em_avail = [c for c in EM if c in all_countries]

print(f'G7 available: {g7_avail}')
print(f'EM available: {em_avail}')

# Returns
idx_cols = [f'{c}_index' for c in all_countries]
ret = df[idx_cols].pct_change().dropna()
ret.columns = [c.replace('_index', '') for c in ret.columns]

ret_g7 = ret[[c for c in g7_avail if c in ret.columns]]
ret_em = ret[[c for c in em_avail if c in ret.columns]]

print(f'\nData: {ret.shape[0]} days')
print(f'Period: {ret.index.min().date()} to {ret.index.max().date()}')

In [ ]:
# === CRISIS CATALOG ===
CRISES = {
    'Lehman': pd.Timestamp('2008-09-15'),
    'Flash_Crash': pd.Timestamp('2010-05-06'),
    'Euro_Crisis': pd.Timestamp('2011-08-05'),
    'Taper_Tantrum': pd.Timestamp('2013-05-22'),
    'China_Crash': pd.Timestamp('2015-08-24'),
    'Brexit': pd.Timestamp('2016-06-24'),
    'Volmageddon': pd.Timestamp('2018-02-05'),
    'COVID': pd.Timestamp('2020-03-11'),
    'Gilt_Crisis': pd.Timestamp('2022-09-23'),
    'SVB': pd.Timestamp('2023-03-10')
}
CRISES = {k: v for k, v in CRISES.items() if v >= ret.index.min()}
print(f'Crises in range: {len(CRISES)}')

---
# PART 1: CRISIS FACTOR (La señal base)

In [ ]:
# === CRISIS FACTOR (global) ===
def compute_cf(r, window=20):
    cf = []
    for i in range(window, len(r)):
        wr = r.iloc[i-window:i]
        corr = wr.corr().values
        n = len(corr)
        avg_corr = (corr.sum() - n) / (n * (n - 1))
        avg_vol = wr.std().mean()
        cf.append(avg_corr * avg_vol * 100)
    return pd.Series(cf, index=r.index[window:])

CF = compute_cf(ret)
CF_G7 = compute_cf(ret_g7)
CF_EM = compute_cf(ret_em)

print(f'CF computed: Global, G7, EM')

---
# PART 2: TEMPORAL DECOMPOSITION (3 Bandas)

In [ ]:
# === 3-BAND DECOMPOSITION ===
def decompose_3bands(series, wavelet='db4'):
    coeffs = pywt.wavedec(series.values, wavelet, level=6)
    
    # Fast (<10d)
    fast_c = [np.zeros_like(c) for c in coeffs]
    for i in range(min(3, len(coeffs))):
        fast_c[i] = coeffs[i]
    fast = pywt.waverec(fast_c, wavelet)[:len(series)]
    
    # Medium (10-60d)
    med_c = [np.zeros_like(c) for c in coeffs]
    for i in range(3, min(5, len(coeffs))):
        med_c[i] = coeffs[i]
    medium = pywt.waverec(med_c, wavelet)[:len(series)]
    
    # Slow (>60d)
    slow_c = [np.zeros_like(c) for c in coeffs]
    for i in range(5, len(coeffs)):
        slow_c[i] = coeffs[i]
    slow = pywt.waverec(slow_c, wavelet)[:len(series)]
    
    return {
        'fast': pd.Series(fast, index=series.index),
        'medium': pd.Series(medium, index=series.index),
        'slow': pd.Series(slow, index=series.index)
    }

bands = decompose_3bands(CF)
print('3-Band decomposition: Fast, Medium, Slow')

---
# PART 3: GEOGRAPHIC RELATIVITY (Δ_geo)

In [ ]:
# === GEOGRAPHIC DISPERSION ===
def compute_geographic_dispersion(ret_g7, ret_em, window=60):
    """Δ_geo = |σ²_G7 - σ²_EM| (spatial decoupling)"""
    var_g7 = ret_g7.var(axis=1).rolling(window).mean()
    var_em = ret_em.var(axis=1).rolling(window).mean()
    delta_geo = (var_g7 - var_em).abs()
    return delta_geo, var_g7, var_em

DELTA_GEO, VAR_G7, VAR_EM = compute_geographic_dispersion(ret_g7, ret_em)

# Visualize
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

ax1 = axes[0]
ax1.plot(VAR_G7.index, VAR_G7.values, 'b-', label='G7', linewidth=1)
ax1.plot(VAR_EM.index, VAR_EM.values, 'orange', label='EM', linewidth=1)
ax1.set_ylabel('Variance (long scale)')
ax1.set_title('Temporal Relativity: G7 vs Emerging Markets (60-250d scale)')
ax1.legend()

ax2 = axes[1]
diff = VAR_G7 - VAR_EM
ax2.fill_between(diff.index, diff.values, 0, where=diff > 0, 
                 alpha=0.5, color='blue', label='G7 higher')
ax2.fill_between(diff.index, diff.values, 0, where=diff <= 0, 
                 alpha=0.5, color='orange', label='EM higher')
ax2.axhline(0, color='black', linewidth=0.5)
ax2.set_ylabel('G7 - EM')
ax2.set_title('Which group perceives risk earlier?')
ax2.legend()

for ax in axes:
    for name, date in CRISES.items():
        if date > VAR_G7.index.min() and date < VAR_G7.index.max():
            ax.axvline(date, color='gray', alpha=0.3, linestyle=':')

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/Caria/great_caria_v3_geographic.png', dpi=150)
plt.show()

---
# PART 4: THE CARIA UNCERTAINTY PRINCIPLE (H)

$$H_{total} = H_{temp} + \lambda \cdot \Delta_{geo}$$

**H alto** = sistema desacoplado = robusto
**H → 0** = sincronización = CRISIS

In [ ]:
# === PHYSICS-FIRST WEIGHTS ===
WEIGHTS = {
    'fast': 0.15,   # Noise
    'medium': 0.35, # Resonance (critical)
    'slow': 0.50    # Structural
}

print('Temporal weights:')
for k, v in WEIGHTS.items():
    print(f'  {k}: {v:.0%}')

In [ ]:
# === COMPUTE μ_MSFI (Mean State) ===
def normalize(s):
    return (s - s.mean()) / (s.std() + 1e-8)

# Normalize bands
s_fast = normalize(bands['fast'])
s_medium = normalize(bands['medium'])
s_slow = normalize(bands['slow'])

# μ = weighted mean
MU = (WEIGHTS['fast'] * s_fast + 
      WEIGHTS['medium'] * s_medium + 
      WEIGHTS['slow'] * s_slow)

print(f'μ_MSFI computed: mean={MU.mean():.3f}, std={MU.std():.3f}')

In [ ]:
# === COMPUTE H_temp (Temporal Uncertainty) ===
def compute_H_temp(s_fast, s_medium, s_slow, mu, weights):
    """H_temp = sqrt(Σ w_i (s_i - μ)²)"""
    H = np.sqrt(
        weights['fast'] * (s_fast - mu)**2 +
        weights['medium'] * (s_medium - mu)**2 +
        weights['slow'] * (s_slow - mu)**2
    )
    return H

H_TEMP = compute_H_temp(s_fast, s_medium, s_slow, MU, WEIGHTS)
print(f'H_temp computed: mean={H_TEMP.mean():.3f}')

In [ ]:
# === COMPUTE H_total ===
# H_total = H_temp + λ * Δ_geo

# Align indices
common_idx = MU.index.intersection(DELTA_GEO.dropna().index)

# Normalize Δ_geo to same scale as H_temp
delta_geo_norm = normalize(DELTA_GEO.loc[common_idx])
h_temp_aligned = H_TEMP.loc[common_idx]
mu_aligned = MU.loc[common_idx]

# Lambda = sensitivity to geographic decoupling
LAMBDA = 0.3  # Can be tuned

H_TOTAL = h_temp_aligned + LAMBDA * delta_geo_norm.abs()
H_TOTAL = H_TOTAL.clip(lower=0.01)  # Avoid division by zero

print(f'H_total computed: mean={H_TOTAL.mean():.3f}')

---
# PART 5: THE PROBABILITY CONE

In [ ]:
# === PROBABILITY CONE ===
# Upper/Lower = μ ± k * H_total
K = 2.0  # Number of standard deviations

UPPER = mu_aligned + K * H_TOTAL
LOWER = mu_aligned - K * H_TOTAL

# The CRISIS INDICATOR: Inverse of H (low H = high danger)
# When H collapses, it means synchronization
SYNC_DANGER = 1 / (H_TOTAL + 0.1)  # Inverse H

print(f'Probability cone computed')
print(f'Sync Danger range: [{SYNC_DANGER.min():.2f}, {SYNC_DANGER.max():.2f}]')

In [ ]:
# === THE CARIA UNCERTAINTY VISUALIZATION ===
fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

# Panel A: Multi-Scale Dynamics
ax1 = axes[0]
ax1.plot(s_slow.loc[common_idx].index, s_slow.loc[common_idx].values, 
         'b-', linewidth=1.5, label='Slow (Structure)')
ax1.plot(s_medium.loc[common_idx].index, s_medium.loc[common_idx].values, 
         color='gold', linewidth=1, label='Medium (Resonance)')
ax1.plot(s_fast.loc[common_idx].index, s_fast.loc[common_idx].values, 
         'gray', alpha=0.5, linewidth=0.5, label='Fast (Noise)')
ax1.set_ylabel('Stress Level')
ax1.set_title('A. Multi-Scale Dynamics (Dissociated vs. Synchronized)')
ax1.legend(loc='upper left')

# Panel B: The Probability Cone
ax2 = axes[1]
ax2.fill_between(mu_aligned.index, LOWER.values, UPPER.values, 
                 alpha=0.3, color='pink', label='Heisenberg Uncertainty (H)')
ax2.plot(mu_aligned.index, mu_aligned.values, 'darkred', 
         linewidth=1.5, label='MSFI (Mean State)')
ax2.set_ylabel('Systemic Fragility')
ax2.set_title('B. The Caria Uncertainty Principle: Probability Cone')
ax2.legend(loc='upper left')

# Panel C: Synchronization Danger (1/H)
ax3 = axes[2]
ax3.fill_between(SYNC_DANGER.index, SYNC_DANGER.values, alpha=0.5, color='red')
ax3.plot(SYNC_DANGER.index, SYNC_DANGER.values, 'darkred', linewidth=0.5)
threshold = SYNC_DANGER.quantile(0.90)
ax3.axhline(threshold, color='orange', linestyle='--', label=f'Danger threshold (90th pct)')
ax3.set_ylabel('1/H (Sync Danger)')
ax3.set_title('C. Synchronization Danger (When H collapses → Crisis)')
ax3.legend()

# Add crisis markers
for ax in axes:
    for name, date in CRISES.items():
        if date in common_idx or (date > common_idx.min() and date < common_idx.max()):
            ax.axvline(date, color='blue', alpha=0.3, linestyle=':')

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/Caria/great_caria_v3_uncertainty.png', dpi=200)
plt.show()

---
# PART 6: THE INTEGRATED FRAGILITY INDEX v3

$$IFI_{v3} = \mu_{MSFI} \times \frac{1}{H_{total}} = \frac{\mu}{H}$$

This combines:
- **High μ** = high stress
- **Low H** = synchronization

Both conditions must be met for crisis

In [ ]:
# === IFI v3: μ / H ===
# Raw: μ * (1/H)
# But we want high values = danger, so:
# If μ > 0: divide by H (low H = high danger)
# We use a modified formula to handle negative μ

# Shift μ to be positive (add offset)
mu_positive = mu_aligned - mu_aligned.min() + 0.1

# IFI = μ * (1/H) = μ / H
IFI_v3 = mu_positive / H_TOTAL

# Normalize to [0, 1]
IFI_v3_norm = (IFI_v3 - IFI_v3.min()) / (IFI_v3.max() - IFI_v3.min())

print(f'IFI v3 computed: range=[{IFI_v3_norm.min():.3f}, {IFI_v3_norm.max():.3f}]')

In [ ]:
# === VALIDATE IFI v3 ===

# Crisis labels
def create_crisis_labels(index, crises, pre=20, post=10):
    labels = pd.Series(0, index=index)
    for name, date in crises.items():
        start = date - pd.Timedelta(days=pre)
        end = date + pd.Timedelta(days=post)
        labels[(labels.index >= start) & (labels.index <= end)] = 1
    return labels

y_true = create_crisis_labels(IFI_v3_norm.index, CRISES)

# Compare models
cf_aligned = CF.loc[IFI_v3_norm.index]
cf_norm = (cf_aligned - cf_aligned.min()) / (cf_aligned.max() - cf_aligned.min())

models = {
    'IFI v3 (μ/H)': IFI_v3_norm,
    'CF Only': cf_norm,
    '1/H (Sync Danger)': (SYNC_DANGER - SYNC_DANGER.min()) / (SYNC_DANGER.max() - SYNC_DANGER.min())
}

# ROC
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax1 = axes[0]
auc_results = {}
for name, score in models.items():
    fpr, tpr, _ = roc_curve(y_true, score)
    roc_auc = auc(fpr, tpr)
    auc_results[name] = roc_auc
    ax1.plot(fpr, tpr, label=f'{name} (AUC={roc_auc:.3f})', linewidth=2)

ax1.plot([0, 1], [0, 1], 'k--')
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate')
ax1.set_title('ROC Curves: IFI v3 vs Baselines')
ax1.legend(loc='lower right')
ax1.grid(True, alpha=0.3)

# AUC bar
ax2 = axes[1]
colors = ['#10b981' if v == max(auc_results.values()) else '#6b7280' 
          for v in auc_results.values()]
ax2.barh(list(auc_results.keys()), list(auc_results.values()), color=colors)
ax2.axvline(0.5, color='red', linestyle='--')
ax2.set_xlabel('AUC')
ax2.set_xlim(0.4, 0.9)
ax2.set_title('AUC Comparison')

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/Caria/great_caria_v3_roc.png', dpi=150)
plt.show()

print('\n=== AUC Results ===')
for name, score in sorted(auc_results.items(), key=lambda x: -x[1]):
    print(f'{name:20s}: {score:.3f}')

In [ ]:
# === FINAL VISUALIZATION ===
fig, axes = plt.subplots(4, 1, figsize=(14, 14), sharex=True)

# A. IFI v3
ax1 = axes[0]
ax1.fill_between(IFI_v3_norm.index, IFI_v3_norm.values, alpha=0.5, color='red')
ax1.plot(IFI_v3_norm.index, IFI_v3_norm.values, 'darkred', linewidth=0.5)
ax1.axhline(IFI_v3_norm.quantile(0.75), color='orange', linestyle='--', label='Warning (75%)')
ax1.axhline(IFI_v3_norm.quantile(0.95), color='red', linestyle='--', label='Critical (95%)')
ax1.set_ylabel('IFI v3')
ax1.set_title(f'A. Integrated Fragility Index v3 (μ/H) - AUC={auc_results.get("IFI v3 (μ/H)", 0):.3f}')
ax1.legend(loc='upper right')

# B. Uncertainty H
ax2 = axes[1]
ax2.fill_between(H_TOTAL.index, H_TOTAL.values, alpha=0.5, color='purple')
ax2.plot(H_TOTAL.index, H_TOTAL.values, 'purple', linewidth=0.5)
ax2.set_ylabel('H (Uncertainty)')
ax2.set_title('B. System Uncertainty (High = Decoupled = Robust)')

# C. Geographic Delta
ax3 = axes[2]
ax3.fill_between(DELTA_GEO.loc[common_idx].index, 
                 DELTA_GEO.loc[common_idx].values, 
                 alpha=0.5, color='blue')
ax3.set_ylabel('Δ_geo')
ax3.set_title('C. Geographic Decoupling |G7 - EM|')

# D. S&P 500 with warnings
ax4 = axes[3]
if 'USA_index' in df.columns:
    sp500 = df['USA_index'].loc[common_idx]
    ax4.plot(sp500.index, sp500.values, 'k-', linewidth=0.5)
    
    # Add danger zones
    danger_threshold = IFI_v3_norm.quantile(0.90)
    danger_periods = IFI_v3_norm > danger_threshold
    ax4.fill_between(IFI_v3_norm.index, sp500.min(), sp500.max(),
                     where=danger_periods, alpha=0.3, color='red')

ax4.set_ylabel('S&P 500')
ax4.set_title('D. Market with Danger Zones (IFI > 90th pct)')
ax4.set_yscale('log')

# Crisis lines
for ax in axes:
    for name, date in CRISES.items():
        if date > common_idx.min() and date < common_idx.max():
            ax.axvline(date, color='blue', alpha=0.3, linestyle=':')

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/Caria/great_caria_v3_final.png', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
# === PRE-CRISIS ANALYSIS ===
print('\n=== Pre-Crisis Analysis (20 days before) ===')

def get_pre_value(series, date, days=20):
    pre = series[(series.index < date) & 
                 (series.index > date - pd.Timedelta(days=days))]
    return pre.mean() if len(pre) > 0 else np.nan

results = []
for name, date in CRISES.items():
    if date < common_idx.min() + pd.Timedelta(days=200) or date > common_idx.max():
        continue
    
    results.append({
        'Crisis': name,
        'Date': date.date(),
        'IFI_v3': get_pre_value(IFI_v3_norm, date),
        'H_total': get_pre_value(H_TOTAL, date),
        'CF': get_pre_value(cf_norm, date)
    })

results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

In [ ]:
# === EXPORT ===
import json
from datetime import datetime

# Current state
current = {
    'ifi_v3': float(IFI_v3_norm.iloc[-1]),
    'mu': float(mu_aligned.iloc[-1]),
    'H_total': float(H_TOTAL.iloc[-1]),
    'H_temp': float(h_temp_aligned.iloc[-1]),
    'delta_geo': float(DELTA_GEO.iloc[-1]),
    'cf': float(CF.iloc[-1])
}

# Status based on IFI v3
if current['ifi_v3'] >= IFI_v3_norm.quantile(0.95):
    status = 'CRITICAL'
elif current['ifi_v3'] >= IFI_v3_norm.quantile(0.75):
    status = 'WARNING'
else:
    status = 'STABLE'

export = {
    'version': 'Great Caria v3.0 (General Financial Relativity)',
    'generated': datetime.now().isoformat(),
    'theory': 'IFI = μ/H where μ=mean fragility, H=uncertainty (temporal + geographic)',
    'status': status,
    'current': current,
    'thresholds': {
        'warning': float(IFI_v3_norm.quantile(0.75)),
        'critical': float(IFI_v3_norm.quantile(0.95))
    },
    'auc_scores': auc_results,
    'crises_validated': len(results_df),
    'interpretation': {
        'high_H': 'System decoupled (G7/EM divergent, scales divergent) = ROBUST',
        'low_H': 'System synchronized (all aligned) = CRISIS IMMINENT'
    }
}

with open('/content/drive/MyDrive/Caria/great_caria_v3.json', 'w') as f:
    json.dump(export, f, indent=2)

print('\n' + '='*60)
print('GREAT CARIA v3.0 - FINAL SUMMARY')
print('='*60)
print(json.dumps(export, indent=2))